In [44]:
from gensim.models import word2vec


# Settings
seed = 5201314
sg = 0
window_size = 25
vector_size = 300
min_count = 2
workers = 60
epochs = 100
batch_words = 400

train_data = word2vec.LineSentence('../training_data/process_content.txt')
model = word2vec.Word2Vec(
    train_data,
    min_count=min_count,
    vector_size=vector_size,
    workers=workers,
    epochs=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words
)

model.save('word2vec.model')

In [45]:
model = word2vec.Word2Vec.load('word2vec.model')
for item in model.wv.most_similar('鏡頭'):
    print(item)

('冷調', 0.16146089136600494)
('便宜', 0.15765829384326935)
('名字', 0.15757183730602264)
('讚嘆', 0.15454351902008057)
('綠色', 0.1512092798948288)
('bionic', 0.15001274645328522)
('找到', 0.1460101455450058)
('方法', 0.14369849860668182)
('抽獎', 0.14083178341388702)
('神經', 0.14052189886569977)


In [46]:
import pandas as pd
score_list = []
raw_content = pd.read_csv('../training_data/Content_cut.csv')
content_list = eval(raw_content.loc[0,'content_cut'])
for i in content_list:
    try:
        score = model.wv.similarity('電池', i)
        score_list.append([score, i])
    except:
        continue
score_df = pd.DataFrame(data = score_list, columns=['Score', 'Word'])
score_df = score_df.drop_duplicates(subset=['Score', 'Word'], keep=False)
score_df.sort_values(by=['Score'], ascending=False)

,Score,Word
2073,0.048014,18日
4416,0.030655,cpu
6876,0.030060,soc
4114,0.027233,ipad
4474,0.013239,2016年
7076,0.008693,wifi6
5470,-0.014296,home
3271,-0.014680,俗
6267,-0.033520,mah
6203,-0.070448,led
